# ***🦜 Chatbot - RAG OIC 🦜***


In [ ]:
%%capture
!pip install pandas openpyxl langchain openai langchain-openai langchain-community langchain-core langchain-text-splitters chromadb pypdf
!pip install streamlit

In [ ]:
%%writefile app_final_proyect.py

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run /content/app_final_proyect.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501